# Parse Experimental Results & Generate Latex Table

In [1]:
% matplotlib inline
import os, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_dir = 'data/data-recsys16'

In [3]:
dat_suffix = ['Osak', 'Glas', 'Edin', 'Toro', 'Melb']
dat_names = ['Osaka', 'Glasgow', 'Edinburgh', 'Toronto', 'Melbourne']

In [4]:
dat_types = ['all', 'nofew']

In [5]:
methods = ['PersTour', 'RankP', 'RankF', 'MC-DP', 'MC-ILP', 'Prop-DP', 'Prop-ILP', 'CRF', 'CRF1']

In [6]:
F1_ijcai_mean = [0.686, 0.801, 0.656, 0.720, 0]
F1_ijcai_std  = [0.233, 0.214, 0.223, 0.215, 0]

### Latex Table for Dataset Statistics

In [7]:
strs = []
for dset in dat_types:
    if dset == 'all':     
        title = 'of all trajectories without loops'
        noloop = True
    if dset == 'nofew':   
        title = 'of users with more than 5 (including 5) trajectories with loops'
        noloop = False
    strs.append('\\begin{table*}\n')
    strs.append('\\centering\n')
    strs.append('\\caption{Dataset ' + title + '}\n')
    strs.append('\\label{table:data:' + dset + '}\n')
    #strs.append('\\small\n')
    strs.append('\\begin{tabular}{' + 'l' + 5*'r' + '} \\hline\n')
    strs.append('\\textbf{City} & \\textbf{\\#POIs} & \\textbf{\\#Users} & ')
    strs.append('\\textbf{\\#POI Visits} & \\textbf{\\#Trajectories} & \\textbf{\\#TotalNodes} \\\\ \\hline\n')
    
    for dat_ix in range(len(dat_names)):
        if noloop == True:
            ftraj = os.path.join(data_dir, 'traj-noloop-' + dset + '-' + dat_suffix[dat_ix] + '.csv')
        else:
            ftraj = os.path.join(data_dir, 'traj-' + dset + '-' + dat_suffix[dat_ix] + '.csv')
        traj_df = pd.read_csv(ftraj)
        total_nodes = traj_df[['trajID', 'trajLen']].copy().groupby('trajID').first().sum().values[0]
        strs.append(dat_names[dat_ix])
        strs.append(' & ' + '{:,}'.format(traj_df['poiID'].unique().shape[0]))
        strs.append(' & ' + '{:,}'.format(traj_df['userID'].unique().shape[0]))
        strs.append(' & ' + '{:,}'.format(traj_df['#photo'].sum()))
        strs.append(' & ' + '{:,}'.format(traj_df['trajID'].unique().shape[0]))
        strs.append(' & ' + '{:,}'.format(total_nodes))
        strs.append(' \\\\ \n')
    strs.append('\\hline\n')
    strs.append('\\end{tabular}\n')
    strs.append('\\end{table*}\n\n')
print(''.join(strs))

\begin{table*}
\centering
\caption{Dataset of all trajectories without loops}
\label{table:data:all}
\begin{tabular}{lrrrrr} \hline
\textbf{City} & \textbf{\#POIs} & \textbf{\#Users} & \textbf{\#POI Visits} & \textbf{\#Trajectories} & \textbf{\#TotalNodes} \\ \hline
Osaka & 27 & 450 & 7,747 & 1,115 & 1,372 \\ 
Glasgow & 27 & 601 & 11,434 & 2,227 & 2,749 \\ 
Edinburgh & 28 & 1,454 & 33,944 & 5,028 & 7,853 \\ 
Toronto & 29 & 1,395 & 39,419 & 6,057 & 7,607 \\ 
Melbourne & 85 & 1,000 & 23,995 & 5,106 & 7,246 \\ 
\hline
\end{tabular}
\end{table*}

\begin{table*}
\centering
\caption{Dataset of users with more than 5 (including 5) trajectories with loops}
\label{table:data:nofew}
\begin{tabular}{lrrrrr} \hline
\textbf{City} & \textbf{\#POIs} & \textbf{\#Users} & \textbf{\#POI Visits} & \textbf{\#Trajectories} & \textbf{\#TotalNodes} \\ \hline
Osaka & 24 & 40 & 2,471 & 473 & 577 \\ 
Glasgow & 25 & 94 & 7,116 & 1,433 & 1,711 \\ 
Edinburgh & 28 & 192 & 19,149 & 2,901 & 4,420 \\ 
Toronto & 29 & 2

### Latex Table for Recommendation Results

Generate results filenames.

In [8]:
def gen_fname(data_dir, dat_suffix, dat_ix, dat_types, type_ix, noloop, uspecific, alpha, C, KX):
    assert(0 <= dat_ix < len(dat_suffix))
    assert(0 <= type_ix < len(dat_types))
    assert(isinstance(noloop, bool))
    assert(isinstance(uspecific, bool))
    assert(0 < alpha < 1)
    alpha_str = str(alpha).replace('.', '_') + '-'
    C_str = 'C' + str(C) + '-'
    KX_str = str(KX) + 'X-'
    
    if noloop == True:
        loop_str = 'noloop-'
    else:
        loop_str = ''
    
    type_str = dat_types[type_ix] + '-'
    
    if uspecific == True:
        user_str = 'specific-'
        suffix = KX_str + dat_suffix[dat_ix] + '.pkl'
    else:
        user_str = 'agnostic-'
        suffix = dat_suffix[dat_ix] + '.pkl'
    
    fname = loop_str + type_str + user_str
    frank = os.path.join(data_dir, 'rank-' + fname + suffix)
    ftran = os.path.join(data_dir, 'tran-' + fname + suffix)
    fcomb = os.path.join(data_dir, 'comb-' + fname + alpha_str + suffix)
    fcrf  = os.path.join(data_dir, 'crf-' + fname + C_str + suffix)
    fcrf1  = os.path.join(data_dir, 'crf1-' + fname + C_str + suffix)
    return frank, ftran, fcomb, fcrf, fcrf1

In [9]:
gen_fname(data_dir, dat_suffix, 1, dat_types, 1, False, False, 0.5, 1, 100)

('data/data-recsys16/rank-nofew-agnostic-Glas.pkl',
 'data/data-recsys16/tran-nofew-agnostic-Glas.pkl',
 'data/data-recsys16/comb-nofew-agnostic-0_5-Glas.pkl',
 'data/data-recsys16/crf-nofew-agnostic-C1-Glas.pkl',
 'data/data-recsys16/crf1-nofew-agnostic-C1-Glas.pkl')

Compute the F1 score for recommended trajectory.

In [10]:
def calc_F1(seq_act, seq_rec):
    '''Compute recall, precision and F1 when trajectories contain sub-tours'''
    assert(len(seq_act) > 0)
    assert(len(seq_rec) > 0)
    match_tags = np.zeros(len(seq_act), dtype=np.bool)
    for poi in seq_rec:
        for j in range(len(seq_act)):
            if match_tags[j] == False and poi == seq_act[j]:
                match_tags[j] = True
                break
    intersize = np.nonzero(match_tags)[0].shape[0]
    recall = intersize / len(seq_act)
    precision = intersize / len(seq_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

Load results data.

In [27]:
def load_results(dat_ix, type_ix, alpha, C, KX, uspecific):
    assert(0 <= dat_ix < len(dat_suffix))
    assert(0 <= type_ix < len(dat_types))
    assert(0 < alpha < 1)
    assert(isinstance(uspecific, bool))
    
    if uspecific == True:
        noloop = False
    else:
        noloop = True
    
    frank, ftran, fcomb, fcrf, fcrf1 = gen_fname(data_dir, dat_suffix, dat_ix, dat_types, type_ix, \
                                                 noloop, uspecific, alpha, C, KX)
    #print(frank)
    assert(os.path.exists(frank))
    #print(ftran)
    assert(os.path.exists(ftran))
    #print(fcomb)
    assert(os.path.exists(fcomb))
    #print(fcrf)
    assert(os.path.exists(fcrf))
    #print(fcrf1)
    assert(os.path.exists(fcrf1))

    # load results data
    recdict_rank = pickle.load(open(frank, 'rb'))
    recdict_tran = pickle.load(open(ftran, 'rb'))
    recdict_comb = pickle.load(open(fcomb, 'rb'))
    recdict_crf  = pickle.load(open(fcrf,  'rb'))
    recdict_crf1 = pickle.load(open(fcrf1, 'rb'))
    
    return recdict_rank, recdict_tran, recdict_comb, recdict_crf, recdict_crf1

Calculate F1-scores from loaded results.

In [12]:
def calc_metrics(recdict_rank, recdict_tran, recdict_comb, recdict_crf, recdict_crf1):
    # compute F1
    F1_rank1 = []  # rank pop
    F1_rank2 = []  # rank feature
    for key in sorted(recdict_rank.keys()):
        F1_rank1.append(calc_F1(recdict_rank[key]['REAL'], recdict_rank[key]['REC_POP']))
        F1_rank2.append(calc_F1(recdict_rank[key]['REAL'], recdict_rank[key]['REC_FEATURE']))
    
    F1_tran1 = []  # transition DP
    F1_tran2 = []  # transition ILP
    for key in sorted(recdict_tran.keys()):
        F1_tran1.append(calc_F1(recdict_tran[key]['REAL'], recdict_tran[key]['REC_DP']))
        F1_tran2.append(calc_F1(recdict_tran[key]['REAL'], recdict_tran[key]['REC_ILP']))

    F1_comb1 = []  # combine rank and transition DP
    F1_comb2 = []  # combine rank and transition ILP
    for key in sorted(recdict_comb.keys()):
        F1_comb1.append(calc_F1(recdict_comb[key]['REAL'], recdict_comb[key]['REC_DP']))
        F1_comb2.append(calc_F1(recdict_comb[key]['REAL'], recdict_comb[key]['REC_ILP']))
        
    F1_crf = []   # structured prediction
    for key in sorted(recdict_crf.keys()):
        F1_crf.append(calc_F1(recdict_crf[key]['REAL'], recdict_crf[key]['REC_CRF']))
    
    F1_crf1 = []   # structured prediction
    for key in sorted(recdict_crf1.keys()):
        F1_crf1.append(calc_F1(recdict_crf1[key]['REAL'], recdict_crf1[key]['REC_CRF']))
    
    # compute mean and std of F1
    F1_mean = [np.mean(x) for x in [F1_rank1, F1_rank2, F1_tran1, F1_tran2, F1_comb1, F1_comb2, F1_crf, F1_crf1]]
    F1_std  = [np.std(x)  for x in [F1_rank1, F1_rank2, F1_tran1, F1_tran2, F1_comb1, F1_comb2, F1_crf, F1_crf1]]
    
    return F1_mean, F1_std

Generate Latex tables from calculated metrics.

In [13]:
def gen_latex_table(F1mean_df, F1std_df, ismax_df, type_ix, uspecific):
    assert(isinstance(uspecific, bool))
    if dat_types[type_ix] == 'all': 
        dstr = 'of all trajectories without loops'
        ustr = 'user agnostic setting'
    if dat_types[type_ix] == 'nofew': 
        dstr = 'of users with more than 5 (including 5) trajectories with loops'
        ustr = 'user specific setting'

    strs = []
    strs.append('\\begin{table*}\n')
    strs.append('\\centering\n')
    strs.append('\\caption{Experimental Results: ' + ustr + ' ' + dstr + '}\n')
    #strs.append('\\small\n')
    strs.append('\\begin{tabular}{l|' + (F1mean_df.shape[1])*'c' + '} \\hline\n')
    for col in F1mean_df.columns:
        strs.append(' & ' + col)
    strs.append(' \\\\ \\hline\n')
    for ix in F1mean_df.index:
        for j in range(F1mean_df.shape[1]):
            if j == 0: strs.append(ix + ' ')
            jx = F1mean_df.columns[j]
            strs.append('& $')
            if ismax_df.loc[ix, jx] == True: strs.append('\\mathbf{')
            strs.append('%.3f' % F1mean_df.loc[ix, jx] + '\\pm' + '%.3f' % F1std_df.loc[ix, jx])
            if ismax_df.loc[ix, jx] == True: strs.append('}')
            strs.append('$ ')
        strs.append('\\\\\n')
    strs.append('\\hline\n')
    strs.append('\\end{tabular}\n')
    strs.append('\\end{table*}\n')
    return ''.join(strs)

In [14]:
KX = 100  # 100 folds in user specific setting
alpha = 0.5
C = 1

In [28]:
for type_ix in range(len(dat_types)):
    F1mean_df = pd.DataFrame(data=np.zeros((len(methods), len(dat_names)), dtype=np.float), \
                                  columns=dat_names, index=methods)
    F1std_df  = pd.DataFrame(data=np.zeros((len(methods), len(dat_names)), dtype=np.float), \
                                  columns=dat_names, index=methods)
    
    if dat_types[type_ix] == 'all':
        uspecific = False
    if dat_types[type_ix] == 'nofew':
        uspecific = True
        
    for dat_ix in range(len(dat_suffix)):
        recdict_rank, recdict_tran, recdict_comb, recdict_crf, recdict_crf1 = load_results(dat_ix, type_ix, \
                                                                                           alpha, C, KX, uspecific)
        F1mean, F1std = calc_metrics(recdict_rank, recdict_tran, recdict_comb, recdict_crf, recdict_crf1)
        
        assert(len(F1mean) == len(F1std) == len(methods)-1)
        F1mean_df[dat_names[dat_ix]] = [F1_ijcai_mean[dat_ix]] + F1mean
        F1std_df[dat_names[dat_ix]] = [F1_ijcai_std[dat_ix]] + F1std
        
    ismax_df  = pd.DataFrame(data=np.zeros(F1mean_df.shape, dtype=np.bool), \
                             columns=F1mean_df.columns, index=F1mean_df.index)
    for col in ismax_df.columns:
        maxix = F1mean_df[col].argmax()
        ismax_df.loc[maxix, col] = True
        
    strs = gen_latex_table(F1mean_df, F1std_df, ismax_df, type_ix, uspecific)
    
    print(strs)

\begin{table*}
\centering
\caption{Experimental Results: user agnostic setting of all trajectories without loops}
\begin{tabular}{l|ccccc} \hline
 & Osaka & Glasgow & Edinburgh & Toronto & Melbourne \\ \hline
PersTour & $0.686\pm0.233$ & $\mathbf{0.801\pm0.214}$ & $0.656\pm0.223$ & $0.720\pm0.215$ & $0.000\pm0.000$ \\
RankP & $0.663\pm0.125$ & $0.744\pm0.165$ & $0.701\pm0.160$ & $0.678\pm0.121$ & $0.607\pm0.143$ \\
RankF & $0.679\pm0.113$ & $0.775\pm0.168$ & $0.693\pm0.154$ & $0.752\pm0.167$ & $0.616\pm0.142$ \\
MC-DP & $0.680\pm0.157$ & $0.716\pm0.168$ & $0.628\pm0.172$ & $0.661\pm0.157$ & $0.558\pm0.179$ \\
MC-ILP & $0.706\pm0.150$ & $0.734\pm0.169$ & $0.678\pm0.148$ & $0.688\pm0.138$ & $0.582\pm0.152$ \\
Prop-DP & $0.699\pm0.168$ & $0.738\pm0.176$ & $0.646\pm0.174$ & $0.690\pm0.171$ & $0.690\pm0.171$ \\
Prop-ILP & $0.717\pm0.158$ & $0.762\pm0.170$ & $0.688\pm0.153$ & $0.726\pm0.152$ & $0.726\pm0.152$ \\
CRF & $0.686\pm0.124$ & $0.721\pm0.174$ & $0.645\pm0.166$ & $0.711\pm0.178$ & $0

### Hyperparameters Tuning Results

NOTE: **POI popularity** used here is defined as *the number of distinct users* that visited the POI, which is not affected by user specific upsampling of trajectories.

In [ ]:
dstype = dat_types[2]
dat_ix = 1

In [ ]:
KXs = [1, 2, 4, 8, 10, 20, 50, 100]
ALPHAs = [.1, .2, .3, .4, .5, .6, .7, .8, .9]

Methods based on ranking and transition matrix.

In [ ]:
methods_F1mean = np.zeros((4, len(KXs)), dtype=np.float)
methods_F1std = np.zeros((4, len(KXs)), dtype=np.float)
alphastr = str(0.5).replace('.', '_') + '-'
for j in range(len(KXs)):
    kx = KXs[j]
    kxstr = str(kx) + 'X-'
    recdict_rank, recdict_tran, recdict_comb = load_results(dstype, dat_ix, kxstr, alphastr, uspecific=True)
    F1mean, F1std = calc_metrics(recdict_rank, recdict_tran, recdict_comb)
    for method_ix in [0, 1, 2, 3]:
        methods_F1mean[method_ix, j] = F1mean[method_ix]
        methods_F1std[method_ix, j] = F1std[method_ix]

plt.figure(figsize=[8, 6])
plt.suptitle('Dataset: %s' % datnames[dat_ix], y=0.95, fontsize=12)
for k in [0, 1, 2, 3]:
    ax = plt.subplot(2, 2, k+1)
    plt.errorbar(KXs, methods_F1mean[k], yerr=methods_F1std[k])
    maxix = np.argmax(methods_F1mean[k])
    plt.plot(KXs[maxix], methods_F1mean[k, maxix], marker='o', markersize=10, markerfacecolor='m', markeredgewidth=0)
    plt.xlim([0.9, 109])
    plt.ylim([0, 1.0])
    plt.title('%s, Best_F1: %.3f' % (methods[k], methods_F1mean[k, maxix]), y=0.1)
    plt.xscale('log')
    if k > 1: plt.xlabel('Folds')
    if k % 2 == 0: plt.ylabel('F1')

Methods combine ranking with transition matrix.

In [ ]:
for method_ix in [4, 5]:
    method_F1mean = np.zeros((len(ALPHAs), len(KXs)), dtype=np.float)
    method_F1std = np.zeros((len(ALPHAs), len(KXs)), dtype=np.float)
    for i in range(len(ALPHAs)):
        alpha = ALPHAs[i]
        alphastr = str(alpha).replace('.', '_') + '-'
        for j in range(len(KXs)):
            kx = KXs[j]
            kxstr = str(kx) + 'X-'
            recdict_rank, recdict_tran, recdict_comb = load_results(dstype, dat_ix, kxstr, alphastr, uspecific=True)
            F1mean, F1std = calc_metrics(recdict_rank, recdict_tran, recdict_comb)
            method_F1mean[i, j] = F1mean[method_ix]
            method_F1std[i, j] = F1std[method_ix]

    plt.figure(figsize=[12, 9])
    plt.suptitle('Dataset: %s, Method: %s' % (datnames[dat_ix], methods[method_ix]), y=0.95, fontsize=12)
    for k in range(len(ALPHAs)):
        ax = plt.subplot(3, 3, k+1)
        plt.errorbar(KXs, method_F1mean[k], yerr=method_F1std[k])
        maxix = np.argmax(method_F1mean[k])
        plt.plot(KXs[maxix], method_F1mean[k, maxix], marker='o', markersize=10, markerfacecolor='m', markeredgewidth=0)
        plt.xlim([0.9, 109])
        plt.ylim([0, 1.0])
        plt.title('$\\alpha$ = %.1f, Best_F1: %.3f' % (ALPHAs[k], method_F1mean[k, maxix]), y=0.1)
        plt.xscale('log')
        if k > 5: plt.xlabel('Folds')
        if k % 3 == 0: plt.ylabel('F1')